## Load the data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt


In [2]:
daily_attendance = pd.read_excel("estimated daily attendance.xlsx",sheet_name="Running Total")
exhibit = pd.read_excel("exhibit template.xlsx",sheet_name="Exhibition feature")
exhibit.drop(["Responsible person","SE Attendance","Uptake rate","topic popularity", \
"location in the museum","Exhibit Ranking","Number of other exhibits","Market budget"],inplace=True,axis=1)
time_series = pd.read_excel("month.xlsx",sheet_name="month")


In [3]:
# Basic information about the datast
exhibit_58 = exhibit.loc[:57,]
exhibit_58.head(1)


,exhibit name,start date,end date,duration,General Attendance,Daily General Attendance,country,region,material/medium,material exclusitivity,size,exclusivity,Stories/Not,Timeline type,artists popularity,Market Budget Flag,Construction Period Flag
0,TANABE CHIKUUNSAI\nCONNECTION,2019-05-31,2019-08-25,86,38453.0,447.127907,Japanese,Japan,bamboo,1.0,1.0,0.0,1.0,Contemporary,Not Popular,Low,1.0


In [4]:
exhibit_58.describe()


,duration,General Attendance,Daily General Attendance,material exclusitivity,size,exclusivity,Stories/Not,Construction Period Flag
count,58.000000,58.000000,58.000000,58.000000,40.000000,58.000000,58.000000,58.000000
mean,147.672414,90219.068966,628.753324,0.103448,49.300000,0.344828,0.465517,0.224138
std,91.732905,63318.023887,288.130408,0.307203,54.176918,0.479463,0.503166,0.420655
min,5.000000,3595.000000,272.904110,0.000000,1.000000,0.000000,0.000000,0.000000
25%,86.000000,41403.250000,485.064615,0.000000,7.750000,0.000000,0.000000,0.000000
50%,101.000000,59815.500000,569.925652,0.000000,27.500000,0.000000,0.000000,0.000000
75%,233.000000,131894.500000,693.553301,0.000000,66.000000,1.000000,1.000000,0.000000
max,382.000000,262310.000000,2326.336842,1.000000,200.000000,1.000000,1.000000,1.000000


In [5]:
exhibit_58.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 17 columns):
exhibit name                58 non-null object
start date                  58 non-null datetime64[ns]
end date                    58 non-null datetime64[ns]
duration                    58 non-null int64
General Attendance          58 non-null float64
Daily General Attendance    58 non-null float64
country                     54 non-null object
region                      53 non-null object
material/medium             57 non-null object
material exclusitivity      58 non-null float64
size                        40 non-null float64
exclusivity                 58 non-null float64
Stories/Not                 58 non-null float64
Timeline type               57 non-null object
artists popularity          58 non-null object
Market Budget Flag          58 non-null object
Construction Period Flag    58 non-null float64
dtypes: datetime64[ns](2), float64(7), int64(1), object(7)
memory usage: 7.8

## Fill in missing value

In [6]:
exhibit_58.loc[:,"country"] = exhibit_58.loc[:,"country"].fillna("None")
exhibit_58.loc[:,"region"] =  exhibit_58.loc[:,"region"].fillna("None")
exhibit_58.loc[:,"material/medium"] = exhibit_58.loc[:,"material/medium"].fillna("None")
exhibit_58.loc[:,"size_missing"] = np.array(exhibit_58.loc[:,"size"].isnull()).astype(int)
exhibit_58.loc[:,"size"] = exhibit.loc[:,"size"].fillna(0)
exhibit_58.loc[:,"Timeline type"] = exhibit_58.loc[:,"Timeline type"].fillna("None")

/Users/lush/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/lush/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [7]:
exhibit_58.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 18 columns):
exhibit name                58 non-null object
start date                  58 non-null datetime64[ns]
end date                    58 non-null datetime64[ns]
duration                    58 non-null int64
General Attendance          58 non-null float64
Daily General Attendance    58 non-null float64
country                     58 non-null object
region                      58 non-null object
material/medium             58 non-null object
material exclusitivity      58 non-null float64
size                        58 non-null float64
exclusivity                 58 non-null float64
Stories/Not                 58 non-null float64
Timeline type               58 non-null object
artists popularity          58 non-null object
Market Budget Flag          58 non-null object
Construction Period Flag    58 non-null float64
size_missing                58 non-null int64
dtypes: datetime64[ns](2), flo

## Transform the exhibit-based data into time-series-based data

In [8]:
exhibit_58.head(1)

,exhibit name,start date,end date,duration,General Attendance,Daily General Attendance,country,region,material/medium,material exclusitivity,size,exclusivity,Stories/Not,Timeline type,artists popularity,Market Budget Flag,Construction Period Flag,size_missing
0,TANABE CHIKUUNSAI\nCONNECTION,2019-05-31,2019-08-25,86,38453.0,447.127907,Japanese,Japan,bamboo,1.0,1.0,0.0,1.0,Contemporary,Not Popular,Low,1.0,0


In [9]:
exhibit_58.columns

Index(['exhibit name', 'start date', 'end date', 'duration',
       'General Attendance', 'Daily General Attendance', 'country', 'region',
       'material/medium', 'material exclusitivity', 'size', 'exclusivity',
       'Stories/Not', 'Timeline type', 'artists popularity',
       'Market Budget Flag', 'Construction Period Flag', 'size_missing'],
      dtype='object')

In [10]:
# Add columns to the time-series dataframe
for column in ["region","country","Timeline type","artists popularity","Market Budget Flag","Construction Period Flag","size_missing"]:
    column_dummy_list = list(exhibit_58[column].unique())
    for column_dummy in column_dummy_list:
        column_dummy = str(column_dummy)
        column_dummy_name = column + "_" + column_dummy
        time_series.insert(len(time_series.columns),column_dummy_name,0)

In [11]:
time_series.insert(len(time_series.columns),"duration_avg",0)
time_series.insert(len(time_series.columns),"size_avg",0)
time_series.insert(len(time_series.columns),"exclusivity_sum",0)
time_series.insert(len(time_series.columns),"story_sum",0)


In [12]:
exhibit_58.columns

Index(['exhibit name', 'start date', 'end date', 'duration',
       'General Attendance', 'Daily General Attendance', 'country', 'region',
       'material/medium', 'material exclusitivity', 'size', 'exclusivity',
       'Stories/Not', 'Timeline type', 'artists popularity',
       'Market Budget Flag', 'Construction Period Flag', 'size_missing'],
      dtype='object')

In [13]:
time_series.columns

Index(['month', 'month_id', 'region_Japan', 'region_China', 'region_Korea',
       'region_South Asia',
       'region_The Himalayas and the Tibetan Buddhist World', 'region_None',
       'region_Southeast Asia', 'region_The Persian World and West Asia',
       'region_Entire Asia', 'country_Japanese', 'country_Chinese',
       'country_Korean', 'country_Indian', 'country_Tibetan',
       'country_Various', 'country_Philippine', 'country_Pakistan',
       'country_None', 'country_Arabia', 'country_Asian', 'country_Persian',
       'country_Indonesian', 'Timeline type_Contemporary',
       'Timeline type_Pre-Modern', 'Timeline type_Modern/Contemporary',
       'Timeline type_Modern',
       'Timeline type_Pre-Modern/Modern and Contemporary',
       'Timeline type_None', 'Timeline type_Pre-Modern/Modern',
       'artists popularity_Not Popular', 'artists popularity_Popular',
       'artists popularity_No Artist', 'artists popularity_popular',
       'artists popularity_No Artist ', 'Mark

In [14]:
# Function to transform np.datetime64 to dt.datetime
def to_datetime(date):
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return dt.utcfromtimestamp(timestamp)

In [15]:
for ts_index in range(len(time_series)):
    duration_list = []
    sum_duration = 0
    size_list = []
    sum_size = 0
    exclusivity_list = []
    sum_exclusivity = 0
    story_list = []
    sum_story = 0
    for e_index in range(len(exhibit_58)):
        
        if (time_series.loc[ts_index:ts_index,'month'].values >= exhibit_58.loc \
        [e_index:e_index,"start date"].values)[0] & (time_series.loc[ts_index:ts_index,'month'].values \
        <= exhibit_58.loc[e_index:e_index,"end date"].values)[0] | \
        (time_series.loc[ts_index:ts_index,'month'].values[0] <= exhibit_58.loc \
        [e_index:e_index,"start date"].values[0]) & (to_datetime(time_series.loc[ts_index:ts_index,'month'].values[0]).month \
        == to_datetime(exhibit_58.loc[e_index:e_index,"end date"].values[0]).month):
             
            # Add duration of each exhibit into the list
            duration_list.append(exhibit_58.loc[e_index:e_index,"duration"])
            # Add country into the dummy columns
            country_tmp = "country_" + exhibit_58.loc[e_index:e_index,"country"]
            time_series.loc[ts_index:ts_index,country_tmp] += 1
            # Add region into the dummy columns
            region_tmp = "region_" + exhibit_58.loc[e_index:e_index,"region"]
            time_series.loc[ts_index:ts_index,region_tmp] += 1
            # Add size of each exhibit into the list
            size_list.append(exhibit_58.loc[e_index:e_index,"size"])
            # Add exclisivity of each exhibit into the list
            exclusivity_list.append(exhibit_58.loc[e_index:e_index,"exclusivity"])
            # Add story or not of each exhibit into the list
            story_list.append(exhibit_58.loc[e_index:e_index,"Stories/Not"])
            # Add Timeline type into the dummy columns
            timeline_tmp = "Timeline type_" + exhibit_58.loc[e_index:e_index,"Timeline type"].values[0]
            time_series.loc[ts_index:ts_index,timeline_tmp] += 1
            # Add artist popularity into the dummy columns
            artist_popularity_tmp = "artists popularity_" + exhibit_58.loc[e_index:e_index,"artists popularity"].values[0]
            time_series.loc[ts_index:ts_index,artist_popularity_tmp] += 1
            # Add Market Budget Flag into the dummy columns
            market_tmp = "Market Budget Flag_" + str(exhibit_58.loc[e_index:e_index,"Market Budget Flag"].values[0])
            time_series.loc[ts_index:ts_index,market_tmp] += 1
            # Add Construction Period Flag into the dummy columns
            construction_tmp = "Construction Period Flag_" + str(exhibit_58.loc[e_index:e_index,"Construction Period Flag"].values[0])
            time_series.loc[ts_index:ts_index,construction_tmp] += 1
            # Add size_missing into the dummy columns
            size_missing_tmp = "size_missing_" + str(exhibit_58.loc[e_index:e_index,"size_missing"].values[0])
            time_series.loc[ts_index:ts_index,size_missing_tmp] += 1
    # Add numeric variable's value into the time-series dataframe
    if len(duration_list) != 0:
        for i in range(len(duration_list)):
            sum_duration += duration_list[i].values[0]
        time_series.loc[ts_index:ts_index,"duration_avg"] = sum_duration/len(duration_list)
    else: 
        time_series.loc[ts_index:ts_index,"duration_avg"] = 0        
    if len(size_list) != 0:
        for i in range(len(size_list)):
            sum_size += size_list[i].values[0]
        time_series.loc[ts_index:ts_index,"size_avg"] = sum_size/len(size_list)
    else:
        time_series.loc[ts_index:ts_index,"size_avg"] = 0
    for i in range(len(exclusivity_list)):
        sum_exclusivity += exclusivity_list[i].values[0]
    time_series.loc[ts_index:ts_index,"exclusivity_sum"] = sum_exclusivity
    for i in range(len(story_list)):
        sum_story += story_list[i].values[0]
    time_series.loc[ts_index:ts_index,"story_sum"] = sum_story

            

In [16]:
time_series.head(10)

,month,month_id,region_Japan,region_China,region_Korea,region_South Asia,region_The Himalayas and the Tibetan Buddhist World,region_None,region_Southeast Asia,region_The Persian World and West Asia,...,Market Budget Flag_Medium,Market Budget Flag_High,Construction Period Flag_1.0,Construction Period Flag_0.0,size_missing_0,size_missing_1,duration_avg,size_avg,exclusivity_sum,story_sum
0,2012-11-01,1,1,1,0,2,0,0,0,0,...,0,0,0,4,3,1,276.500000,10.750000,1.0,2.0
1,2012-12-01,2,0,0,1,2,0,0,1,0,...,0,0,2,2,3,1,153.250000,9.750000,2.0,2.0
2,2013-01-01,3,0,0,1,1,0,0,1,1,...,2,1,0,4,3,1,104.250000,111.500000,0.0,2.0
3,2013-02-01,4,4,0,1,1,0,0,1,0,...,0,1,1,7,4,4,136.000000,36.625000,2.0,5.0
4,2013-03-01,5,0,3,1,0,0,0,2,0,...,0,1,1,5,3,3,123.166667,7.833333,2.0,3.0
5,2013-04-01,6,1,1,0,0,1,1,1,0,...,0,1,0,5,2,3,176.400000,5.400000,2.0,1.0
6,2013-05-01,7,3,2,1,2,0,1,1,0,...,1,3,2,8,7,3,103.600000,40.900000,3.0,4.0
7,2013-06-01,8,0,0,0,2,0,0,0,0,...,0,0,0,3,1,2,199.333333,10.000000,0.0,2.0
8,2013-07-01,9,1,1,0,1,0,1,0,0,...,0,1,2,3,2,3,169.800000,20.800000,1.0,3.0
9,2013-08-01,10,3,3,0,0,0,0,0,0,...,1,1,3,4,5,2,137.428571,27.857143,1.0,4.0


In [17]:
time_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 47 columns):
month                                                  82 non-null datetime64[ns]
month_id                                               82 non-null int64
region_Japan                                           82 non-null int64
region_China                                           82 non-null int64
region_Korea                                           82 non-null int64
region_South Asia                                      82 non-null int64
region_The Himalayas and the Tibetan Buddhist World    82 non-null int64
region_None                                            82 non-null int64
region_Southeast Asia                                  82 non-null int64
region_The Persian World and West Asia                 82 non-null int64
region_Entire Asia                                     82 non-null int64
country_Japanese                                       82 non-null int64
country_Chin

In [18]:
time_series.to_csv("time_series.csv")